In [ ]:
# Imports
import rasterio
import time
import numpy as np
import os
import h5py
import cv2

from rasterio.windows import Window
from skimage.io import imread
from PIL import Image

In [ ]:
# DATA TO CHANGE !!!

tileIteratorStart = 0                                                                                                                                                                 
tileNumber = 0

DEMpath = 'data/TerrainPreparedData/Alpy/alpy_raw_' + str(tileNumber) + '_1801.tif'
TIFFpath = 'data/TerrainPreparedData/Alpy/alpy_txt_' + str(tileNumber) + '_1801.tif'

DEMSavePath = 'heightmapsDEM_Alpy_256'
TIFFSavePath = 'heightmapsTIFF_Alpy_256'

tileSize = 1801

# Size of tiles
xSize, ySize = 256, 256

#crop shift
xShift, yShift = 128, 128

In [ ]:
def comparator(heightmap_chunk):
    heightmap_chunk = ((heightmap_chunk - heightmap_chunk.min()) * (1/(heightmap_chunk.max() - heightmap_chunk.min()) * 255)).astype('uint8')
    black_ratio = ((heightmap_chunk <= 40).sum()*1.0) /float(np.prod(heightmap_chunk.shape))

    if black_ratio > 0.55:
        return False
    return True

In [ ]:
iterator = tileIteratorStart

if not os.path.exists(TIFFSavePath):
    os.makedirs(TIFFSavePath)
    
if not os.path.exists(DEMSavePath):
    os.makedirs(DEMSavePath)
    
Image.MAX_IMAGE_PIXELS = None
    
t0 = time.time()

demData = rasterio.open(DEMpath)
tifData = rasterio.open(TIFFpath)

for y in range(0, demData.width, yShift):
    for x in range(0, demData.height, xShift):

        if x > (tileSize - xShift) or y > (tileSize - yShift):
            continue
                   
        cropTextureX = x+xSize
        cropTextureY = y+ySize

        iterator += 1
 
        windowDEM = Window(x, y, xSize, ySize)
        transformDEM = demData.window_transform(windowDEM)
        
        windowTIFF = Window(x, y, xSize, ySize)
        transformTIFF = tifData.window_transform(windowTIFF)

        profileDEM = demData.profile
        profileDEM.update({
            'height': xSize,
            'width': ySize,
            'transform': transformDEM})
        
        profileTIFF = tifData.profile
        profileTIFF.update({
            'height': xSize,
            'width': ySize,
            'transform': transformTIFF})
        
        DEMSavePathName = DEMSavePath + '/topography_' + str(iterator) + '.tif'
        TIFFSavePathName = TIFFSavePath + '/topography_' + str(iterator) + '.tif'
        
        with rasterio.open(DEMSavePathName, 'w', **profileDEM) as dstDEM:
            dstDEM.write(demData.read(window=windowDEM))
        
        with rasterio.open(TIFFSavePathName, 'w', **profileTIFF) as dstTIFF:
            dstTIFF.write(tifData.read(window=windowTIFF))

        TIFFTexture = imread(TIFFSavePathName)

        if not comparator(TIFFTexture):
            os.remove(TIFFSavePathName)
            os.remove(DEMSavePathName)
            iterator -= 1

print(str(time.time()-t0) + 's for ' + TIFFSavePath + '. Saved ' + str(iterator) + ' images')